In [1]:
version = "1.0.0"

# Read the Journals which we are searching for


In [2]:
import pandas as pd

journals = pd.read_csv('doaj_journals.csv', sep="|")
import re
journals.tail

<bound method NDFrame.tail of                                      title  \
0            Ain Shams Engineering Journal   
1                       Biomedical Journal   
2    African Journal of Emergency Medicine   
3           Alexandria Engineering Journal   
4        Anais Brasileiros de Dermatologia   
..                                     ...   
103  Epidemiology and Psychiatric Sciences   
104             Epidemiology and Infection   
105                           BJPsych Open   
106            Forum of Mathematics, Sigma   
107               Forum of Mathematics, Pi   

                                publisher      eissn      pissn  \
0                                Elsevier        NaN  2090-4479   
1                                Elsevier  2320-2890  2319-4170   
2                                Elsevier  2211-419X        NaN   
3                                Elsevier        NaN  1110-0168   
4    Sociedade Brasileira de Dermatologia        NaN  0365-0596   
..             

# Get the article details from DOAJ


In [3]:
def findjournaldata(results):
    publisher = None
    eissn = None
    pissn = None
    title = None
    doajid = None
    categories = None
    language = None
    notfound = True
    # iterate over the result and try to find where title exactly match
    for result in results:
            if result['bibjson']['title'].lower().strip() == journals.iloc[idx]['title'].lower().strip():
                notfound = False
                publisher = result['bibjson']['publisher']['name'] 
                if 'eissn' in result['bibjson'].keys():
                    eissn = result['bibjson']['eissn'] 
                if 'pissn' in result['bibjson'].keys():
                    pissn = result['bibjson']['pissn'] 

                title = result['bibjson']['title'] 
                doajid = result['id'] 
                categories =  "#".join(  [ s['term'].strip() for s in result['bibjson']['subject'] ] ) 
                language = "#".join(  [ s.strip() for s in result['bibjson']['language'] ] ) 
                
                break
    return notfound, publisher, eissn, pissn, title, doajid, categories, language

In [4]:
import requests
import urllib.parse
from time import sleep

publishers = []
eissns = []
pissns = []
titles = []
doajids = []
categoriess = []
languages = []
for idx in range(len(journals)):
    if 'doajid' not in journals.columns or journals.iloc[idx]['doajid'] is None or not isinstance(journals.iloc[idx]['doajid'], str):
        searchterm = urllib.parse.quote('bibjson.title:"'+journals.iloc[idx]['title']+'"')
        searchterm = 'https://doaj.org/api/v2/search/journals/'+searchterm
        x = requests.get(searchterm)
        notfound = True
        if x.status_code == 200:
            # iterate over pages
            jsonresult =  x.json()
            if 'total' in jsonresult.keys() :
                for page in range(int(jsonresult['total'])):
                    page = page+1
                    if 'results' in jsonresult.keys():                
                        notfound, publisher, eissn, pissn, title, doajid, categories, language = findjournaldata(jsonresult['results'])
                        if not notfound:
                            publishers.append(publisher)
                            eissns.append(eissn)
                            pissns.append(pissn)
                            titles.append(title)
                            doajids.append(doajid)
                            categoriess.append(categories)
                            languages.append(language)
                            break
                        else:
                            if jsonresult['total'] > 1:
                                # go to next page
                                x = requests.get(searchterm+"?page="+str(page)+"&pageSize=10")
                                print(searchterm+"?page="+str(page)+"&pageSize=10")
                                if x.status_code == 200:
                                    jsonresult =  x.json()
                                else:
                                    break
                
       
        if notfound:
            # test there is new page if yes go there        
            print("Error:", searchterm)
            publishers.append(None)
            eissns.append(None)
            pissns.append(None)
            titles.append(journals.iloc[idx]['title'])
            doajids.append(None)
            categoriess.append(None)
            languages.append(None)
        sleep(3)
    else:
        publishers.append(journals.iloc[idx]['publisher'])
        eissns.append(journals.iloc[idx]['eissn'])
        pissns.append(journals.iloc[idx]['pissn'])
        titles.append(journals.iloc[idx]['title'])
        doajids.append(journals.iloc[idx]['doajid'])
        categoriess.append(journals.iloc[idx]['categories'])
        languages.append(journals.iloc[idx]['language'])
    
# enrich journals
journals['publisher'] = publishers
journals['eissn'] = eissns
journals['pissn'] = pissns
journals['title'] = titles
journals['doajid'] = doajids
journals['categories'] = categoriess
journals['language'] = languages

journals.to_csv('doaj_journals.csv', sep="|", index=False)


In [5]:
print(journals.head())
journals.tail()

                                   title  \
0          Ain Shams Engineering Journal   
1                     Biomedical Journal   
2  African Journal of Emergency Medicine   
3         Alexandria Engineering Journal   
4      Anais Brasileiros de Dermatologia   

                              publisher      eissn      pissn  \
0                              Elsevier        NaN  2090-4479   
1                              Elsevier  2320-2890  2319-4170   
2                              Elsevier  2211-419X        NaN   
3                              Elsevier        NaN  1110-0168   
4  Sociedade Brasileira de Dermatologia        NaN  0365-0596   

                             doajid  \
0  5622b8878c204fd2b88130fbeea2f2d7   
1  d1e768538ffa4f82826f04f163a7fa38   
2  8150ee089d8b4f099f6c30bd8aaf0fcc   
3  40ae382df340487c8266f817e3a03683   
4  bbbc2abf7b4a4a74a81784b75472ca54   

                                          categories language  
0  Engineering (General). Civil engineering (

,title,publisher,eissn,pissn,doajid,categories,language
103,Epidemiology and Psychiatric Sciences,Cambridge University Press,2045-7979,2045-7960,394000a0a14c48c1ad8a371057a89eba,Neurosciences. Biological psychiatry. Neuropsy...,EN
104,Epidemiology and Infection,Cambridge University Press,1469-4409,NaN,a1f2ae4bc11f4ff59a4bd50492e97bcf,Infectious and parasitic diseases#Public aspec...,EN
105,BJPsych Open,Cambridge University Press,2056-4724,NaN,d4970d85595442289b084bfb036f8c1e,Psychiatry,EN
106,"Forum of Mathematics, Sigma",Cambridge University Press,2050-5094,NaN,fdbdb8bd9cd94bb288ce1ec2c4265481,Mathematics,EN
107,"Forum of Mathematics, Pi",Cambridge University Press,2050-5086,NaN,33d4851361894493bf8b1a9d02596c29,Mathematics,EN


# Get the articles for the Journal

In [6]:
def findarticledata(results):
    
    # iterrate over Articles
    url = []
    title = []
    doi = []
    abstract  = []
    writers = []
    publishdate = []
    keyword = []
    for result in results:

        # url
        urlfound = False
        for l in result['bibjson']["link"]:
            if l['type'] == "fulltext":
                urlfound = True
                url.append(  l['url'].replace("\n", "") )
                break
        if not urlfound:
            # if there no url we do not interest about the Article
            url.append(None)
            title.append(None)
            doi.append(None)
            abstract.append( None )
            writers.append(None)
            publishdate.append( None )
            keyword.append( None )
            continue
                
            
        # title
        if "title" in  result['bibjson']:
            title.append( result['bibjson']["title"].replace("\n", "") )
        else:
            # if there no title we do not interest about the Article
            title.append(None)
            doi.append(None)
            abstract.append( None )
            writers.append(None)
            publishdate.append( None )
            keyword.append( None )
            continue
        
        # doi
        founddoi = False
        for ide in result['bibjson']['identifier']:
            if ide["type"].lower() == "doi":
                founddoi = True
                doi.append( ide['id'].replace("\n", "") )
                break
        if not founddoi:
            doi.append(None)
        
        # abstract
        if "abstract" in  result['bibjson']:
            abstract.append( result['bibjson']["abstract"].replace("\n", '<br>') )
        else:
            # if there no abstract we do not interest about the Article
            abstract.append( None )
            writers.append(None)
            publishdate.append( None )
            keyword.append( None )
            continue
        
        # writer
        writer = ""
        for w in result['bibjson']["author"]:
            writer = writer + "#" + w['name'].replace("\n", "")
            if 'affiliation' in w.keys():
                writer = writer + "--" + w['affiliation'].replace("\n", "")
            if 'orcid_id' in w.keys():
                writer = writer + "---" + w['orcid_id'].replace("\n", "")          
        writers.append(writer)
        
        # publishdate
        if "created_date" in result.keys():
            publishdate.append( result["created_date"].replace("\n", "") )
        else:
            publishdate.append( None )
        
        # keywords
        # we put there a filter as if there is too much keyword the are not informatiom
        if "keywords" in result['bibjson'] and len(result['bibjson']["keywords"]) < 15 :
            keytext = ""            
            for k in result['bibjson']["keywords"]:
                if len(k) < 100:
                    keytext = keytext + "#" + k.replace("\n", "")
            if len(keytext) == 0:
                keyword.append( None )
            else:
                keyword.append( keytext )
        else:
            keyword.append( None )

    df = pd.DataFrame({
        'url': url,
        'title': title,
        'doi': doi,
        'abstract': abstract,
        'writer': writers,
        'publishdate': publishdate,
        'keyword': keyword,
                      })
        
    return df

In [ ]:
import csv
import numpy as np

# iterate over the journals
for idx in range(len(journals)): 
    print("load:", journals.iloc[idx]['title'], "Ready: ", str(np.round(100*idx/len(journals),2))+"%", end='\r')
    searchtext = 'journal:"'+journals.iloc[idx]['title']+'" AND _exists_:doi AND _exists_:abstract  AND _exists_: "bibjson.author" '
    # add issn
    if isinstance(journals.iloc[idx]['eissn'], str):
        searchtext = searchtext + ' AND issn: "'+journals.iloc[idx]['eissn']+'"'
    else:
        if isinstance(journals.iloc[idx]['pissn'], str):
            searchtext = searchtext + ' AND issn: "'+journals.iloc[idx]['pissn']+'"'        
    searchterm = urllib.parse.quote(searchtext)
    searchterm = 'https://doaj.org/api/v2/search/articles/'+searchterm
    print(searchterm +'?page=1&pageSize=100&sort=bibjson.title:desc')
    x = requests.get(searchterm +'?page=1&pageSize=100&sort=bibjson.title:desc')
    notfound = True
    
    journaldf  = pd.DataFrame({
        'url': [],
        'title': [],
        'doi': [],
        'abstract': [],
        'writer': [],
        'publishdate': [],
        'keyword': [],
        'journal_title': [],
        'journal_eissn': [],
        'journal_pissn': [],
        'category': []
                      })
    
    if x.status_code == 200:
        # iterate over pages
        jsonresult =  x.json()
        if 'total' in jsonresult.keys() :
            pagenumber = int(jsonresult['total']//100)+1
            if jsonresult['total'] > 1000 :
                print("Can not collect all of the Article for", journals.iloc[idx]['title'], "Total:", jsonresult['total'])
            for page in range(pagenumber):
                if 'results' in jsonresult.keys():   
                    thispage = findarticledata(jsonresult['results'])
                    thispage['journal_title'] = journals.iloc[idx]['title']
                    thispage['journal_eissn'] = journals.iloc[idx]['eissn']
                    thispage['journal_pissn'] = journals.iloc[idx]['pissn']
                    thispage['category'] = journals.iloc[idx]['categories']
                    
                    journaldf = pd.concat([journaldf, thispage])                    
                # go to othe next page
                if jsonresult['total'] > 1 and page != pagenumber-1 :
                    sleep(3)
                    # go to next page
                    x = requests.get(searchterm+"?page="+str(page+2)+"&pageSize=100")
                    if x.status_code == 200:
                        jsonresult =  x.json()
                    else:
                        break
                        
    # save journaldf
    journaldf.to_pickle('data/journal_'+journals.iloc[idx]['title'].replace(" ", "_").replace("&", "and")+".pandas" )    
    sleep(60)

https://doaj.org/api/v2/search/articles/journal%3A%22Ain%20Shams%20Engineering%20Journal%22%20AND%20_exists_%3Adoi%20AND%20_exists_%3Aabstract%20%20AND%20_exists_%3A%20%22bibjson.author%22%20%20AND%20issn%3A%20%222090-4479%22?page=1&pageSize=100&sort=bibjson.title:desc
https://doaj.org/api/v2/search/articles/journal%3A%22Biomedical%20Journal%22%20AND%20_exists_%3Adoi%20AND%20_exists_%3Aabstract%20%20AND%20_exists_%3A%20%22bibjson.author%22%20%20AND%20issn%3A%20%222320-2890%22?page=1&pageSize=100&sort=bibjson.title:desc
https://doaj.org/api/v2/search/articles/journal%3A%22African%20Journal%20of%20Emergency%20Medicine%22%20AND%20_exists_%3Adoi%20AND%20_exists_%3Aabstract%20%20AND%20_exists_%3A%20%22bibjson.author%22%20%20AND%20issn%3A%20%222211-419X%22?page=1&pageSize=100&sort=bibjson.title:desc
https://doaj.org/api/v2/search/articles/journal%3A%22Alexandria%20Engineering%20Journal%22%20AND%20_exists_%3Adoi%20AND%20_exists_%3Aabstract%20%20AND%20_exists_%3A%20%22bibjson.author%22%20%20AN

https://doaj.org/api/v2/search/articles/journal%3A%22Genes%20and%20Diseases%22%20AND%20_exists_%3Adoi%20AND%20_exists_%3Aabstract%20%20AND%20_exists_%3A%20%22bibjson.author%22%20%20AND%20issn%3A%20%222352-3042%22?page=1&pageSize=100&sort=bibjson.title:desc
https://doaj.org/api/v2/search/articles/journal%3A%22Genomics%2C%20Proteomics%20%26%20Bioinformatics%22%20AND%20_exists_%3Adoi%20AND%20_exists_%3Aabstract%20%20AND%20_exists_%3A%20%22bibjson.author%22%20%20AND%20issn%3A%20%221672-0229%22?page=1&pageSize=100&sort=bibjson.title:desc
